# Imports #

In [48]:
import pandas as pd
import numpy as np
import datetime
import time
from decouple import config
import requests
import urllib.parse
import json
AAapikey = config('AAKey')

# Gather Historic Stock News #

In [49]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)

In [50]:
def appendToDf(data):
    try:
        for i in data.get("feed"):
            test_date = i.get("time_published")
            test_date = test_date[:8]
            newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date() 
            row = [newDate, i.get("title"), ticker]
            new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
            #historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
            return new_df
    except:
        print("Request failed")
        print(data)
    #return new_df

In [51]:
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1), interval = 365):
    # Format for news is [Date, Headline, Ticker]
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    url = 'https://www.alphavantage.co/query?'
    window = 15
    startDate = endDate - datetime.timedelta(days = interval)
    windowStart = startDate
    windowEnd = startDate + datetime.timedelta(days = window)
    delta = datetime.timedelta(days = window)
    count = 0
    while (windowEnd <= endDate):
        if count == 5:
            time.sleep(60)
            count = 0
        startDateAA = toAADate(windowStart)
        endDateAA = toAADate(windowEnd)
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDateAA, 'time_to': endDateAA, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
        r = requests.get(url, params = Myparams)
        data = r.json()
        try:
            for i in data.get("feed"):
                test_date = i.get("time_published")
                test_date = test_date[:8]
                newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date() 
                row = [newDate, i.get("title"), ticker]
                new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
                historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
        except:
            print("Request failed")
            print("windowStart is: " +str(windowStart))
            print("windowEnd is: " +str(windowEnd))
            print(data)
        windowStart = windowEnd
        windowEnd = windowEnd + delta
        count = count + 1
    print("Final rolling date is: " +str(windowEnd))
    #print(historic_parsed_news)
    return historic_news


In [52]:

ticker = input('Input Ticker:\n')  
print(ticker)

TSLA


In [53]:
hist = getHistoricNewsData(ticker)

Final rolling date is: 2023-03-03


In [54]:
print(hist.head)

<bound method NDFrame.head of             Date                                           Headline Ticker
0     2022-03-01  Tesla Rival Slashes 2022 Production Outlook; L...   TSLA
1     2022-03-01  Nasdaq moves into positive territory as broade...   TSLA
2     2022-03-01  BYD to use Baidu's autonomous driving technolo...   TSLA
3     2022-03-01  Chinese electric car makers' February sales de...   TSLA
4     2022-03-02  Box Stock Jumps As Fourth-Quarter Results, Out...   TSLA
...          ...                                                ...    ...
4681  2023-02-03                Why Tesla Stock Keeps Racing Higher   TSLA
4682  2023-02-03  Elon Musk surprise appearance as high-profile ...   TSLA
4683  2023-02-03  Bill Gates says Elon Musk not yet a 'great phi...   TSLA
4684  2023-02-03  Market Rally Powers Higher On Tame Fed, Meta: ...   TSLA
4685  2023-02-03  Treasury makes more electric SUVs eligible for...   TSLA

[4686 rows x 3 columns]>


In [55]:
histname = "datasets/" + ticker + "-hist.csv"
hist.to_csv(histname)